In [60]:
import pandas as pd
import xarray as xr
import numpy as np

import random
import warnings


import calliope
from calliope.exceptions import ModelWarning

# Suppress the specific ModelWarning from Calliope
warnings.filterwarnings("ignore", category=ModelWarning)

from deap import base
from deap import creator
from deap import tools

In [69]:
initial_cap =  [4999.999988, 2019.286486, 3427.703196, 2750.418696, 0.003076]

cap_limit = [[0.0, 5000.0], [0.0, 50000.0], [0.0, 50000.0], [0.0, 50000.0], [0.0, 50000.0]]

model = calliope.Model('C:/Users/Jacob/Desktop/PythonProjects/Calliope with miniforge3/GAMGA-Calliope v3.9/GAMGA_model/model.yaml')

model.run()

df_total_cost = model.results.cost.to_series().dropna()
total_cost_optimal = df_total_cost.loc[~df_total_cost.index.map(str).str.contains('co2_emissions')].sum()

energy_cap_df = model.results.energy_cap.to_pandas()
filtered_energy_cap_df = energy_cap_df[~energy_cap_df.index.str.contains("demand|transmission")]

print(filtered_energy_cap_df)

updates = [
    {'tech': tech, 'loc': loc}
    for loc_tech in filtered_energy_cap_df.index
    for loc, tech in [loc_tech.split("::")]  # Split index by '::' to separate loc and tech
]

c:\Users\Jacob\miniconda3\envs\calliope\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.



loc_techs
region2::solarPV    7886.408026
region2::battery    3017.357403
region1::solarPV    2865.188158
region1::ccgt       4999.915739
region1::battery       9.021193
dtype: float64


In [70]:
total_cost_optimal

optimal_value = total_cost_optimal

max_slack = 0.5

In [ ]:
def update_energy_cap_max_for_individual(model, updates, individual_values):
    
    # Ensure the length of updates matches the individual's values
    if len(updates) != len(individual_values):
        raise ValueError("Length of updates and individual values must match.")
    
    # Update the model with the individual's capacity values
    for update, new_cap in zip(updates, individual_values):
        tech = update['tech']
        loc = update['loc']
        
        # Construct the location::technology key and update the model
        loc_tech_key = f"{loc}::{tech}"
        model.backend.update_param('energy_cap_max', {loc_tech_key: new_cap})
        model.backend.update_param('energy_cap_min', {loc_tech_key: new_cap})
    
    # Run the model for this individual
    try:
        rerun_model = model.backend.rerun()  # Rerun to capture updated backend parameters

        # Calculate the total cost, excluding emission costs
        df_total_cost = rerun_model.results.cost.to_series().dropna()
        total_cost = round(df_total_cost.loc[~df_total_cost.index.map(str).str.contains('co2_emissions')].sum(), 2)

    except Exception as e:
        # If solving fails, set total cost to NaN and print a warning
        total_cost = float('inf')
        print("Warning: Model could not be solved for the individual. Assigning cost as infinite.")
    
    return total_cost

def slack_feasibility(individual):
    cost = update_energy_cap_max_for_individual(model, updates, individual)
    slack_distance = (cost - optimal_value) / optimal_value

    # Update feasibility condition based on the new criteria
    feasible = slack_distance <= max_slack and cost >= optimal_value

    print(f"Slack feasibility for individual: {feasible}, Cost: {cost}")
    
    return feasible 